In [34]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

In [35]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"diwakarmogaralaslu","key":"01f39cff4e7593382e52958c52990706"}'}

In [36]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [37]:
!kaggle datasets download -d tmdb/tmdb-movie-metadata

Dataset URL: https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata
License(s): other
tmdb-movie-metadata.zip: Skipping, found more recently modified local copy (use --force to force download)


In [39]:
!unzip tmdb-movie-metadata.zip

Archive:  tmdb-movie-metadata.zip
replace tmdb_5000_credits.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace tmdb_5000_movies.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [40]:
data = pd.read_csv("tmdb_5000_movies.csv")

In [41]:
sentence=""
for i in data.iloc[3]["genres"].replace(" ",''):
    if i.isalpha()== True:

            sentence +=i


sentence.split("idname")[1:]

['Action', 'Crime', 'Drama', 'Thriller']

In [43]:
for i in range(len(data)):
    sentence = ""
    for j in data.iloc[i]["genres"].replace(" ",''):
        if j.isalpha()== True:

            sentence +=j
    sentence = sentence.replace('idname'," ").strip().split(" ")
    data.at[i,"category"] = sentence

In [44]:
data.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,category
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,"[Action, Adventure, Fantasy, ScienceFiction]"
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"[Adventure, Fantasy, Action]"
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,"[Action, Adventure, Crime]"
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,"[Action, Crime, Drama, Thriller]"
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,"[Action, Adventure, ScienceFiction]"


In [45]:
dam = []
for i in range(len(data)):
    for j in data.loc[i,"category"]:
        dam.append(j)

In [46]:
dam = set(dam)
dam

{'',
 'Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Foreign',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'ScienceFiction',
 'TVMovie',
 'Thriller',
 'War',
 'Western'}

In [47]:
for i in dam:
    if i != "":
        data[i] =  0
    else:
        data["Unknown"] = 0

In [48]:
for i in dam:
    for j in range(len(data)):
        if i != "":
            if i in data.loc[j,"category"]:
                data.at[j,i] = 1
        else:
            if i in data.loc[j,"category"]:
                data.at[j,"None"] = 1

In [49]:
columns_2_delete = ['budget', 'genres', 'homepage', 'id', 'keywords','original_title','popularity', 'production_companies','release_date', 'revenue', 'runtime','spoken_languages', 'status', 'tagline', 'title', 'vote_average','vote_count',]
for column in columns_2_delete:
    del data[column]

In [50]:
import regex as re
def extract_country(s):
    match = re.search(r'"name"\s*:\s*"([^"]+)"', s)
    return match.group(1) if match else None

data["country_name"] = data["production_countries"].apply(extract_country)

In [51]:
data = data[['original_language', 'overview','country_name', 'category',"Unknown", 'Comedy', 'Fantasy', 'Adventure', 'Thriller', 'Mystery','Animation', 'History', 'Romance', 'Foreign', 'Horror', 'Documentary','Family', 'Western', 'Music', 'ScienceFiction', 'War', 'TVMovie','Action', 'Drama', 'Crime', ]]

In [52]:
x = data[["overview","country_name","original_language"]]
y = data.iloc[:,4:]

In [53]:
y

,Unknown,Comedy,Fantasy,Adventure,Thriller,Mystery,Animation,History,Romance,Foreign,...,Documentary,Family,Western,Music,ScienceFiction,War,TVMovie,Action,Drama,Crime
0,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
4799,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4800,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0
4801,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
# prompt: see my output is y so i want to predict all the column names which has 1 in it for the x after tfidf

import numpy as np
import scipy
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(x['overview'])

# Assuming 'y' is your target variable (a DataFrame with columns representing genres and 0/1 values)
# and you want to predict which columns in 'y' have a value of 1 based on the TF-IDF representation of 'overview'.


def predict_genres(tfidf_vector, y_data):
  """Predicts genre columns with value 1 based on TF-IDF vector.
  Args:
      tfidf_vector: TF-IDF vector of the overview.
      y_data: DataFrame with genre columns.
  Returns:
      List of predicted genres
  """
  # Calculate cosine similarity between input text's TF-IDF vector and all other overviews
  cosine_similarities = cosine_similarity(tfidf_vector, X_tfidf)

  # Find the most similar overview
  most_similar_index = np.argmax(cosine_similarities)

  # Get the corresponding genre values from 'y' for the most similar overview
  predicted_genres = y_data.iloc[most_similar_index]

  return predicted_genres[predicted_genres == 1].index.tolist()

# Example Usage:
example_index = 0 # Choose an index to use as an example.
example_tfidf = X_tfidf[example_index]
predicted_genre = predict_genres(example_tfidf, y)
print(f"Predicted genres for the example at index {example_index}: {predicted_genre}")


ValueError: np.nan is an invalid document, expected byte or unicode string.